# Compute AQI

Ref: https://www.kaggle.com/code/rohanrao/calculating-aqi-air-quality-index-tutorial/notebook

In [48]:
## importing packages
import numpy as np
import pandas as pd
import os

In [49]:
directory = "/Users/rnirms/Documents/omdena/data/CPCB_downloads/2023-03-07/csv_latest/"
all_files = os.listdir(directory)
csv_files = [file for file in all_files if file.endswith(".csv")]
csv_files = sorted(csv_files)
print(csv_files)

['BandraKurlaComplexMumbaiIITM.csv', 'BandraMumbaiMPCB.csv', 'BorivaliEastMumbaiIITM.csv', 'BorivaliEastMumbaiMPCB.csv', 'ChakalaAndheriEastMumbaiIITM.csv', 'ChhatrapatiShivajiIntlAirportT2MumbaiMPCB.csv', 'ColabaMumbaiMPCB.csv', 'DeonarMumbaiIITM.csv', 'KandivaliEastMumbaiMPCB.csv', 'KhindipadaBhandupWestMumbaiIITM.csv', 'KurlaMumbaiMPCB.csv', 'MaladWestMumbaiIITM.csv', 'MazgaonMumbaiIITM.csv', 'MulundWestMumbaiMPCB.csv', 'NavyNagarColabaMumbaiIITM.csv', 'PowaiMumbaiMPCB.csv', 'SiddharthNagarWorliMumbaiIITM.csv', 'SionMumbaiMPCB.csv', 'VasaiWestMumbaiMPCB.csv', 'VileParleWestMumbaiMPCB.csv', 'WorliMumbaiMPCB.csv']


In [50]:
df_list = []
station_names = []
for filename in csv_files: 
    df_temp = pd.read_csv(os.path.join(directory, filename))
    df_list.append(df_temp)
    name, _ = os.path.splitext(filename)
    station_names.append(name)
data_dict = dict(zip(station_names, df_list))

In [51]:
def clean_data(data):
    data = data.iloc[:, 2:-1] # Remove columns: Unnamed:0, From Date, Unnamed:23
    data = data.rename(columns={'To Date': 'Date'}) # Rename date column
    data["Date"] = pd.to_datetime(data.Date, format='%d-%m-%Y %H:%M') # Format Date column as datetime
    data = data.replace('None', np.NaN) # Replace all 'None' to NaN
    data[data.columns[1:]] = data[data.columns[1:]].astype(float) # Format all other columns to float 

    return data 


In [52]:
for key in data_dict.keys():
    data_dict[key] = clean_data(data_dict[key])

In [53]:
data_dict['BandraKurlaComplexMumbaiIITM'].columns

Index(['Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'SO2', 'CO',
       'Ozone', 'Benzene', 'Toluene', 'Eth-Benzene', 'MP-Xylene', 'RH', 'WS',
       'WD', 'SR', 'BP', 'Xylene', 'AT', 'RF', 'TOT-RF'],
      dtype='object')

In [54]:
data_dict['BandraKurlaComplexMumbaiIITM'].head()

,Date,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone,...,MP-Xylene,RH,WS,WD,SR,BP,Xylene,AT,RF,TOT-RF
0,2021-01-01 00:15:00,147.26,175.93,16.23,61.72,45.30,79.76,4.68,1.38,13.0,...,NaN,94.14,0.46,187.53,NaN,953.2,0.0,21.84,0.0,0.0
1,2021-01-01 00:30:00,145.37,173.96,16.88,61.50,45.75,79.01,18.49,1.38,15.0,...,NaN,94.99,0.47,224.11,NaN,953.2,0.0,21.84,0.0,0.0
2,2021-01-01 00:45:00,156.71,186.12,22.71,62.99,51.15,84.96,17.62,1.27,10.0,...,NaN,93.88,0.46,212.47,NaN,953.2,0.0,22.01,0.0,0.0
3,2021-01-01 01:00:00,171.18,205.41,16.55,59.13,44.19,77.83,18.14,1.23,12.0,...,NaN,93.38,0.27,216.65,NaN,954.2,0.0,21.99,0.0,0.0
4,2021-01-01 01:15:00,183.17,219.31,20.39,59.87,47.66,81.63,18.69,1.24,14.0,...,NaN,92.44,0.17,103.12,NaN,954.9,0.0,22.04,0.0,0.0


## Get pollutant rolling averages 

In [55]:
def get_rolling_average(df):
    if("PM10" in df.columns):
        df["PM10_24hr_avg"] = df["PM10"].rolling(window = 24, min_periods = 16).mean().values
    
    if("PM2.5" in df.columns):
        df["PM2.5_24hr_avg"] = df["PM2.5"].rolling(window = 24, min_periods = 16).mean().values

    if("SO2" in df.columns):    
        df["SO2_24hr_avg"] = df["SO2"].rolling(window = 24, min_periods = 16).mean().values

    if("NOx" in df.columns):    
        df["NOx_24hr_avg"] = df["NOx"].rolling(window = 24, min_periods = 16).mean().values

    if("NH3" in df.columns):
        df["NH3_24hr_avg"] = df["NH3"].rolling(window = 24, min_periods = 16).mean().values
    
    if("CO" in df.columns):
        df["CO_8hr_max"] = df["CO"].rolling(window = 8, min_periods = 1).max().values
    
    if("Ozone" in df.columns):
        df["Ozone_8hr_max"] = df["Ozone"].rolling(window = 8, min_periods = 1).max().values

    return df 

In [56]:
for key in data_dict.keys():
    data_dict[key] = get_rolling_average(data_dict[key])

In [57]:
data_dict['BandraKurlaComplexMumbaiIITM'].columns

Index(['Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'SO2', 'CO',
       'Ozone', 'Benzene', 'Toluene', 'Eth-Benzene', 'MP-Xylene', 'RH', 'WS',
       'WD', 'SR', 'BP', 'Xylene', 'AT', 'RF', 'TOT-RF', 'PM10_24hr_avg',
       'PM2.5_24hr_avg', 'SO2_24hr_avg', 'NOx_24hr_avg', 'NH3_24hr_avg',
       'CO_8hr_max', 'Ozone_8hr_max'],
      dtype='object')

## Compute pollutant sub-index

In [58]:
## PM2.5 Sub-Index calculation
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

In [59]:
for key in data_dict.keys():
    data_dict[key]["PM2.5_SubIndex"] = data_dict[key]["PM2.5_24hr_avg"].apply(lambda x: get_PM25_subindex(x))

In [60]:
## PM10 Sub-Index calculation
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

In [61]:
for key in data_dict.keys():
    data_dict[key]["PM10_SubIndex"] = data_dict[key]["PM10_24hr_avg"].apply(lambda x: get_PM10_subindex(x))

In [62]:
## SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

In [63]:
for key in data_dict.keys():
    if("SO2_24hr_avg" in data_dict[key].columns):
        data_dict[key]["SO2_SubIndex"] = data_dict[key]["SO2_24hr_avg"].apply(lambda x: get_SO2_subindex(x))

In [64]:
## NOx Sub-Index calculation
def get_NOx_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

In [65]:
for key in data_dict.keys():
    if("NOx_24hr_avg" in data_dict[key].columns):
        data_dict[key]["NOx_SubIndex"] = data_dict[key]["NOx_24hr_avg"].apply(lambda x: get_NOx_subindex(x))

In [66]:
## NH3 Sub-Index calculation
def get_NH3_subindex(x):
    if x <= 200:
        return x * 50 / 200
    elif x <= 400:
        return 50 + (x - 200) * 50 / 200
    elif x <= 800:
        return 100 + (x - 400) * 100 / 400
    elif x <= 1200:
        return 200 + (x - 800) * 100 / 400
    elif x <= 1800:
        return 300 + (x - 1200) * 100 / 600
    elif x > 1800:
        return 400 + (x - 1800) * 100 / 600
    else:
        return 0

In [67]:
for key in data_dict.keys():
    if("NH3_24hr_avg" in data_dict[key].columns):
        data_dict[key]["NH3_SubIndex"] = data_dict[key]["NH3_24hr_avg"].apply(lambda x: get_NH3_subindex(x))

In [68]:
## CO Sub-Index calculation
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

In [69]:
for key in data_dict.keys():
    if("CO_24hr_avg" in data_dict[key].columns):
        data_dict[key]["CO_SubIndex"] = data_dict[key]["CO_24hr_avg"].apply(lambda x: get_CO_subindex(x))

In [70]:
## Ozone Sub-Index calculation
def get_Ozone_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

In [71]:
for key in data_dict.keys():
    if("Ozone_24hr_avg" in data_dict[key].columns):
        data_dict[key]["Ozone_SubIndex"] = data_dict[key]["Ozone_24hr_avg"].apply(lambda x: get_CO_subindex(x))

In [72]:
## AQI bucketing
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN

In [73]:
for key in data_dict.keys():
    count = 0 
    df = data_dict[key]
    
    count += (df["PM2.5_SubIndex"] > 0).astype(int)
    count += (df["PM10_SubIndex"] > 0).astype(int)
    if("SO2_SubIndex" in df.columns):
        count += (df["SO2_SubIndex"] > 0).astype(int)
    if("NOx_SubIndex" in df.columns):
        count += (df["NOx_SubIndex"] > 0).astype(int)
    if("NH3_SubIndex" in df.columns):
        count += (df["NH3_SubIndex"] > 0).astype(int)
    if("CO_SubIndex" in df.columns):
        count += (df["CO_SubIndex"] > 0).astype(int)
    if("Ozone_SubIndex" in df.columns):
        count += (df["Ozone_SubIndex"] > 0).astype(int)

    df["Checks"] = count

In [74]:
data_dict['BandraKurlaComplexMumbaiIITM'].columns

Index(['Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'SO2', 'CO',
       'Ozone', 'Benzene', 'Toluene', 'Eth-Benzene', 'MP-Xylene', 'RH', 'WS',
       'WD', 'SR', 'BP', 'Xylene', 'AT', 'RF', 'TOT-RF', 'PM10_24hr_avg',
       'PM2.5_24hr_avg', 'SO2_24hr_avg', 'NOx_24hr_avg', 'NH3_24hr_avg',
       'CO_8hr_max', 'Ozone_8hr_max', 'PM2.5_SubIndex', 'PM10_SubIndex',
       'SO2_SubIndex', 'NOx_SubIndex', 'NH3_SubIndex', 'Checks'],
      dtype='object')

In [75]:
for key in data_dict.keys():
    df = data_dict[key]

    subindex_cols = [col for col in df.columns if 'SubIndex' in col]

    df["AQI_calculated"] = round(df[subindex_cols].max(axis = 1))
    df.loc[df["PM2.5_SubIndex"] + df["PM10_SubIndex"] <= 0, "AQI_calculated"] = np.NaN
    df.loc[df.Checks < 3, "AQI_calculated"] = np.NaN
    df["AQI_bucket_calculated"] = df["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))


In [76]:
data_dict['BandraKurlaComplexMumbaiIITM'].columns

Index(['Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'SO2', 'CO',
       'Ozone', 'Benzene', 'Toluene', 'Eth-Benzene', 'MP-Xylene', 'RH', 'WS',
       'WD', 'SR', 'BP', 'Xylene', 'AT', 'RF', 'TOT-RF', 'PM10_24hr_avg',
       'PM2.5_24hr_avg', 'SO2_24hr_avg', 'NOx_24hr_avg', 'NH3_24hr_avg',
       'CO_8hr_max', 'Ozone_8hr_max', 'PM2.5_SubIndex', 'PM10_SubIndex',
       'SO2_SubIndex', 'NOx_SubIndex', 'NH3_SubIndex', 'Checks',
       'AQI_calculated', 'AQI_bucket_calculated'],
      dtype='object')

In [78]:
df = data_dict['BandraKurlaComplexMumbaiIITM']
df[~df.AQI_calculated.isna()].tail()


,Date,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone,...,CO_8hr_max,Ozone_8hr_max,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NOx_SubIndex,NH3_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
76470,2023-03-08 13:45:00,89.07,187.55,128.94,107.15,163.90,173.35,50.83,0.0,165.0,...,0.02,176.0,195.481944,151.758056,72.715104,232.125833,51.753646,5,232.0,Poor
76471,2023-03-08 14:00:00,93.35,177.91,96.45,105.15,128.26,148.24,43.74,0.0,150.0,...,0.00,176.0,195.408333,151.686111,71.497917,230.913333,51.620729,5,231.0,Poor
76472,2023-03-08 14:15:00,95.64,177.82,118.26,158.14,177.40,179.00,40.37,0.0,192.0,...,0.00,192.0,196.983333,151.575000,69.706771,231.240417,51.801771,5,231.0,Poor
76473,2023-03-08 14:30:00,101.09,175.04,105.69,149.00,163.99,155.00,40.38,0.0,51.0,...,0.00,192.0,198.280556,151.417500,68.098437,229.282917,51.453333,5,229.0,Poor
76474,2023-03-08 14:41:00,92.99,196.95,101.40,159.51,164.63,171.00,42.69,0.0,166.0,...,0.00,192.0,198.395833,151.843889,66.548958,226.830833,51.115521,5,227.0,Poor
